In [1]:
"""
    To use this implementation, you simply have to implement `agent_function` such that it returns a legal action.
    You can then let your agent compete on the server by calling
        python3 client_simple.py path/to/your/config.json
    
    The script will keep running forever.
    You can interrupt it at any time.
    The server will remember the actions you have sent.

    Note:
        By default the client bundles multiple requests for efficiency.
        This can complicate debugging.
        You can disable it by setting `single_request=True` in the last line.
"""
import itertools
import json
import logging
import requests
import time
"""
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    import sys
    run(sys.argv[1], agent_function, single_request=False)
"""

"\nif __name__ == '__main__':\n    logging.basicConfig(level=logging.INFO)\n    import sys\n    run(sys.argv[1], agent_function, single_request=False)\n"

In [2]:
def sum_lists(*args):
    return list(map(sum, zip(*args)))
def divide_list(mylist,integer):
    return list(map(lambda x: x/integer, mylist))

def is_within_bounds(pos):
    x,y=pos
    constraints = {
            -6: {3},
            -5: {2,3},
            -4: range(1,4),
        -3: range(-3,7),
        -2: range(-3,6),
        -1: range(-3,5),
        0: range(-3,4),
        1: range(-4,4),
        2: range(-5,4),
        3: range(-6,4),
        4: range(-3,0),
        5: {-3,-2},
        6:  {-3},
        }

    if x>6 or x<-6:
        return False
    if y>6 or y<-6:
        return False
    if y in constraints and x not in constraints[y]:
        return False
    
    return True

def get_adjacent_directions(peg):
    # Define all possible move directions on the hexagonal board
    directions = [(-1, +1), (-1, 0), (0, -1), (0, +1), (+1, 0), (+1, -1)]
    # Using the peg's current position, calculate the adjacent positions
    x, y = peg
    adjacent_positions = [[x + dx, y + dy] for dx, dy in directions]

    # Assuming we have a function that checks if the position is within the bounds of the board
    adjacent_positions = [pos for pos in adjacent_positions if is_within_bounds(pos)]

    return adjacent_positions

def get_hop_directions(peg):
    # Define all possible move directions on the hexagonal board
    directions = [(-2, +2), (-2, 0), (0, -2), (0, +2), (+2, 0), (+2, -2)]
    # Using the peg's current position, calculate the adjacent positions
    x, y = peg
    hop_positions = [[x + dx, y + dy] for dx, dy in directions]

    # Assuming we have a function that checks if the position is within the bounds of the board
    hop_positions = [pos for pos in hop_positions if is_within_bounds(pos)]

    return hop_positions

In [3]:
def is_empty(direction, all_pegs):
    # Check if the coordinate is occupied by any peg
    return direction not in all_pegs

def can_hop(peg, direction, all_pegs):
    between_peg=divide_list(sum_lists(peg,direction),2)
    if is_empty(between_peg, all_pegs)==False and is_empty(direction, all_pegs)==True:
        return True
    else: 
        return False

def follow_hop_chain(peg, direction, all_pegs):
    path = [peg]
    current_position = peg

    while can_hop(current_position, direction, all_pegs):
        path.append(direction)
        current_position = direction

        new_hop_directions = get_hop_directions(current_position)
        valid_hop_made = False
        for new_direction in new_hop_directions:
            if can_hop(current_position, new_direction, all_pegs) and new_direction != direction and new_direction!=peg and new_direction not in path:
                direction = new_direction
                valid_hop_made = True
                break

        if not valid_hop_made:
            break

    return current_position, path

def is_in_home_area(peg):
    home_area=[[-3,6],[-3,-5],[-2,5],[-3,4],[-2,4],[-1,4]]
    if peg in home_area:
        return True
    else:
        return False

In [4]:
def get_legal_moves(a_pegs, b_pegs, c_pegs):
    legal_moves = []
    if c_pegs==None:
        all_pegs = a_pegs + b_pegs # Combine all pegs for checking
        #opponent_pegs=b_pegs
    else:
        all_pegs = a_pegs + b_pegs + c_pegs  # Combine all pegs for checking
        #opponent_pegs=b_pegs + c_pegs

    for peg in a_pegs:
        adjacent_directions=get_adjacent_directions(peg)
        hop_directions=get_hop_directions(peg)
        # Check for simple moves
        for direction in adjacent_directions:
            if is_empty(direction, all_pegs):
                legal_moves.append([peg, direction])

        # Check for simple hops and hop chains
        for direction in hop_directions:
            if can_hop(peg, direction, all_pegs):
                end_position, path = follow_hop_chain(peg, direction, all_pegs)
                if end_position != peg:  # To ensure it lands on a different space
                    legal_moves.append(path)
            
            """
        # Check for swap moves
        for opponent_peg in opponent_pegs:
            if is_in_home_area(opponent_peg) and not is_in_home_area(peg):
                legal_moves.append([peg, opponent_peg]"""
    return legal_moves

# The helper functions like is_empty, can_hop, follow_hop_chain, can_swap, and move_to
# need to be implemented according to the rules of FAUhalma.


In [5]:
def make_move(position, move, player):
    a_pegs, b_pegs, c_pegs = position
    
    if c_pegs==None:
        new_a_pegs, new_b_pegs,new_c_pegs = list(a_pegs), list(b_pegs), []
    else:
        new_a_pegs, new_b_pegs, new_c_pegs = list(a_pegs), list(b_pegs), list(c_pegs)

    # Extract start and end position from the move
    start, end = move[0], move[-1]

    if player == 'A':
        # Update player A's pegs
        new_a_pegs.remove(start)
        new_a_pegs.append(end)
        
    elif player == 'B':
        # Update player B's pegs
        new_b_pegs.remove(start)
        new_b_pegs.append(end)

    return new_a_pegs, new_b_pegs, new_c_pegs


def heuristic_value(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home):
    # Efficient calculation of sum of distances
    a_distance = sum(manhattan_distance(peg, find_closest_home(peg, a_home)) for peg in a_pegs)
    b_distance = sum(manhattan_distance(peg, find_closest_home(peg, b_home)) for peg in b_pegs)
    c_distance = 0
    if c_pegs is not None:
        c_distance = sum(manhattan_distance(peg, find_closest_home(peg, c_home)) for peg in c_pegs)

    # Adjust the return value based on the number of players
    if c_pegs is None:
        return (b_distance) - a_distance
    else:
        return (b_distance + c_distance) - a_distance

def manhattan_distance(coord1, coord2):
    # Manhattan distance between two coordinates
    x1, y1 = coord1
    x2, y2 = coord2
    return abs(x1 - x2) + abs(y1 - y2)

def find_closest_home(peg, home_positions):
    # Find the closest home position for the given peg
    return min(home_positions, key=lambda home: manhattan_distance(peg, home))

def game_over(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home):
    # The game is over if all pegs of a player are in their home
    #print(c_pegs)
    if c_pegs==[]:
        #print("game_over içi",all(peg in a_home for peg in a_pegs))
        return all(peg in a_home for peg in a_pegs) or all(peg in b_home for peg in b_pegs)
    else:
        return all(peg in a_home for peg in a_pegs) or all(peg in b_home for peg in b_pegs) or all(peg in c_home for peg in c_pegs)


In [11]:
def parse_game_state(game_state):
    # Always extract coordinates for player A
    player_a_pegs = game_state.get('A', [])

    # Check for the presence of player B and C, and extract their coordinates if they exist
    player_b_pegs = game_state.get('B', [])
    player_c_pegs = game_state.get('C', []) if 'C' in game_state else None

    return player_a_pegs, player_b_pegs, player_c_pegs

def minimax(position, depth,alpha, beta, current_player,a_home, b_home, c_home):
    #print("depth:",depth)
    a_pegs, b_pegs, c_pegs = position
    #print("game over mı?:",game_over(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home))
    if game_over(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home) or depth == 0:
        #print("depth 0- heuristic dönüyor",heuristic_value(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home))
        return heuristic_value(a_pegs, b_pegs, c_pegs, a_home, b_home, c_home)
    
    if current_player == 'A':
        #print("min max içi sıra A")
        max_eval = float('-inf')
        for move in get_legal_moves(a_pegs, b_pegs, c_pegs):
            new_position = make_move(position, move, "A")
            eval = minimax(new_position, depth - 1, alpha, beta, 'B', a_home, b_home, c_home)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cut-off
        return max_eval
    else:
        #print("min max içi sıra B")
        min_eval = float('inf')
        for move in get_legal_moves(b_pegs, a_pegs, c_pegs):
            new_position = make_move(position, move, 'B')
            eval = minimax(new_position, depth - 1, alpha, beta, 'A', a_home, b_home, c_home)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

        
def find_best_move(a_pegs, b_pegs, c_pegs, depth, player, a_home, b_home, c_home):
    best_move = None
    alpha = float('-inf')
    beta = float('inf')

    # Generate all legal moves for the player
    moves = get_legal_moves(a_pegs, b_pegs, c_pegs)
    for move in moves:
        # Create a new state by making a move
        new_a_pegs, new_b_pegs, new_c_pegs = make_move((a_pegs, b_pegs, c_pegs), move, player)

        # Evaluate this new state using the Minimax algorithm with Alpha-Beta pruning
        #print("best move minimax içine giriliyor")
        value = minimax((new_a_pegs, new_b_pegs, new_c_pegs), depth - 1, alpha, beta, 'B' if player == 'A' else 'C', a_home, b_home, c_home)
        #print("best move minimax içinden çıkıldı")
        #print("move best",value,alpha,beta)

        if value > alpha:
            alpha = value
            best_move = move
    #print("best move bulundu")
    return best_move

# Make sure to define the get_legal_moves and make_move functions according to the game's rules.

In [12]:
def agent_function(request_dict,home,depth):
    # TOOD: Implement this function in a better way
    print('I got the following request:')
    print(request_dict)
    a_home,b_home,c_home=home
    game_state = request_dict
    a_pegs, b_pegs, c_pegs = parse_game_state(game_state)
    next_move=find_best_move(a_pegs, b_pegs, c_pegs, depth, "A", a_home, b_home, c_home)
    return next_move

In [13]:
class Fauhalma:
    def __init__(self, three_agent=True):
        # Initialize homes for two-player or three-player game
        self.a_home = [[-3, 5], [-2, 5], [-3, 4], [-2, 4], [-1, 4]]
        if three_agent:
            # Three-player game homes
            self.b_home = [[4, -1], [4, -2], [5, -2], [4, -3], [5, -3]]
            self.c_home = [[-3, -1], [-3, 2], [-2, 2], [-2, -3], [-1, -3]]
        else:
            # Two-player game homes
            self.b_home = [[1, -4], [2, -4], [3, -4], [2, -5], [3, -5]]
            self.c_home = []

        self.home=(self.a_home,self.b_home,self.c_home)
        # Other initializations as needed (e.g., current player)
        self.current_player = 'A'

    def run(self,config_file, action_function,depth,single_request=False):
        home=self.home
        logger = logging.getLogger(__name__)

        with open(config_file, 'r') as fp:
            config = json.load(fp)
        
        logger.info(f'Running agent {config["agent"]} on environment {config["env"]}')
        logger.info(f'Hint: You can see how your agent performs at {config["url"]}agent/{config["env"]}/{config["agent"]}')

        actions = []
        for request_number in itertools.count():
            logger.debug(f'Iteration {request_number} (sending {len(actions)} actions)')
            # send request
            response = requests.put(f'{config["url"]}/act/{config["env"]}', json={
                'agent': config['agent'],
                'pwd': config['pwd'],
                'actions': actions,
                'single_request': single_request,
            })
            if response.status_code == 200:
                response_json = response.json()
                for error in response_json['errors']:
                    logger.error(f'Error message from server: {error}')
                for message in response_json['messages']:
                    logger.info(f'Message from server: {message}')

                action_requests = response_json['action-requests']
                if not action_requests:
                    logger.info('The server has no new action requests - waiting for 1 second.')
                    time.sleep(1)  # wait a moment to avoid overloading the server and then try again
                # get actions for next request
                actions = []
                for action_request in action_requests:
                    actions.append({'run': action_request['run'], 'action': action_function(action_request['percept'],home,depth)})
            elif response.status_code == 503:
                logger.warning('Server is busy - retrying in 3 seconds')
                time.sleep(3)  # server is busy - wait a moment and then try again
            else:
                # other errors (e.g. authentication problems) do not benefit from a retry
                logger.error(f'Status code {response.status_code}. Stopping.')
                break


In [16]:
#Two player game-example (1.2.2 depth 3, 1.2.3 depth 6)
game = Fauhalma(three_agent=False) 
game.run("agent-configs/ws2324.1.2.4.json", agent_function, 6)

I got the following request:
{'A': [[-1, 2], [-2, 1], [-1, 4], [-3, 0], [-3, 2]], 'B': [[0, 0], [1, -4], [1, -1], [2, -5], [0, -2]]}


KeyboardInterrupt: 